# Groundtruth file

Notebook used to generate the ground truth according to the position of GPS in post-process.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import importlib
import scripts.theodolite_utils as theodo_u
import warnings
warnings.filterwarnings('ignore')

In [ ]:
theodo_u = importlib.reload(theodo_u)
path = "../data/20220224/"
prefix = "filtered_prediction/f-2-1-1-1-6-0-L"    # Change to GPS predictions, NOT prisms predictions !
linear_interpolation = True
if(linear_interpolation==True):
    P1 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_1.csv"))
    P2 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_2.csv"))
    P3 = np.array(theodo_u.read_prediction_data_Linear_csv_file(path+prefix +"_3.csv"))
else:
    P1 = np.array(theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_1.csv"))
    P2 = np.array(theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_2.csv"))
    P3 = np.array(theodo_u.read_prediction_data_GP_csv_file(path+prefix +"_3.csv"))

In [ ]:
P = np.array([[ 3.16757534, -0.42103498, -0.12806262,  1.        ],
            [ 3.89296346e+00,  1.96285659e-03, -4.14797074e-01,  1.00000000e+00],
            [ 2.89566976,  0.28583646, -0.46077129,  1.        ]]).T
G1 = np.array([ 2.85818819, -0.25079912, -0.77501231,  1.        ]).T
G2 = np.array([ 3.63328716, -0.20827459, -0.67323631,  1.        ]).T
G3 = np.array([ 3.49290871,  0.28817526, -0.64702379,  1.        ]).T

In [ ]:
# Doing a minimization between these not moving points, and the 3D prism coordinates
# Pose_lidar is a list of each rigid transform founded
theodo_u = importlib.reload(theodo_u)
list_gps_time = []
Pose_G1 = []
Pose_G2 = []
Pose_G3 = []
Prism_corrected = []
number = len(P1)
for i in range(0,number):
    Q = np.array([P1[i,1:4], P2[i,1:4], P3[i,1:4]]).T
    Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
    T = theodo_u.point_to_point_minimization(P, Q)
    Pose_G1.append(T@G1)
    Pose_G2.append(T@G2)
    Pose_G3.append(T@G3)
    prism_correct = T@P
    Prism_corrected.append(prism_correct)
    list_gps_time.append(P1[i,0])

In [ ]:
Pose_G1_arr = np.array(Pose_G1)
Pose_G2_arr = np.array(Pose_G2)
Pose_G3_arr = np.array(Pose_G3)
Prism_corrected_arr = np.array(Prism_corrected)
print(Pose_G1_arr[:])
print(Prism_corrected[0][:,0])

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes()
begin = 0
end = -1
ax.plot(Pose_G1_arr[begin:end,0],Pose_G1_arr[begin:end,1], color='black')
ax.plot(Pose_G2_arr[begin:end,0],Pose_G2_arr[begin:end,1], color='black')
ax.plot(Pose_G3_arr[begin:end,0],Pose_G3_arr[begin:end,1], color='black')
ax.plot(Prism_corrected_arr[begin:end,0,0],Prism_corrected_arr[begin:end,1,0], color='red')
ax.plot(Prism_corrected_arr[begin:end,0,1],Prism_corrected_arr[begin:end,1,1], color='green')
ax.plot(Prism_corrected_arr[begin:end,0,2],Prism_corrected_arr[begin:end,1,2], color='blue')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_aspect('equal')
fig.tight_layout() 

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize = (8, 6))
ax = plt.axes(projection ="3d")
ax.scatter3D(Pose_G1_arr[begin:end,0],Pose_G1_arr[begin:end,1],Pose_G1_arr[begin:end,2], color='black')
ax.scatter3D(Prism_corrected_arr[begin:end,0,0],Prism_corrected_arr[begin:end,1,0],Prism_corrected_arr[begin:end,2,0], color='red')
ax.scatter3D(Prism_corrected_arr[begin:end,0,1],Prism_corrected_arr[begin:end,1,1],Prism_corrected_arr[begin:end,2,1], color='green')
ax.scatter3D(Prism_corrected_arr[begin:end,0,2],Prism_corrected_arr[begin:end,1,2],Prism_corrected_arr[begin:end,2,2], color='blue')
ax.scatter3D(0,0,0, color='orange')
ax.set_xlabel("X")
ax.set_ylabel("Y")
fig.tight_layout()

In [ ]:
theodo_u = importlib.reload(theodo_u)
theodo_u.grountruth_GP_gps_convert_for_eval(list_gps_time, Pose_G1, path + "ground_truth/gps_front_predicted.txt")
theodo_u.grountruth_GP_gps_convert_for_eval(list_gps_time, Pose_G2, path + "ground_truth/gps_back_predicted.txt")
theodo_u.grountruth_GP_gps_convert_for_eval(list_gps_time, Pose_G3, path + "ground_truth/gps_middle_predicted.txt")